# Gather Twitter Data

## Import Dependencies

In [11]:
from config import (consumer_key, consumer_secret, access_token, access_token_secret)
from target_users import (target_users)

In [12]:
import  matplotlib.pyplot  as plt
import pandas as pd
import numpy as np
import tweepy
import json
import datetime
import time
import pprint as pprint
import requests
import json

%matplotlib inline

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

## Create Twitter Authorization

In [13]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

## Gather Twitter data through Twitter API and Tweepy

### Create Module to gather Tweets

In [45]:
def twttr_data ():
    #Create lists to hold tweets
    new_tweets = []
    public_tweets = []
    tweet_ids = []
    timestamp = []
    
    
    new_tweets = api.search('BTCUSD', count=100, result_type="recent")
    for x in range(0, len(new_tweets['statuses'])):
        tweet_id = new_tweets['statuses'][x]['id']
        if tweet_id not in tweet_ids:
            tweet_ids.append(new_tweets['statuses'][x]['id'])
            timestamp.append(new_tweets['statuses'][x]['user']['created_at'])
            public_tweets.append(new_tweets['statuses'][x]['text'])
    
        oldest = tweet_id - 1
    
    for x in range(10):
        new_tweets = api.search('BTCUSD', count=100, result_type="recent", max_id = oldest)
        #print(json.dumps(new_tweets['statuses'], indent = 4, sort_keys = True))
        for x in range(0, len(new_tweets['statuses'])):
            tweet_id = new_tweets['statuses'][x]['id']
            #print(tweet_id)
            if tweet_id not in tweet_ids:
                tweet_ids.append(tweet_id)
                timestamp.append(new_tweets['statuses'][x]['user']['created_at'])
                public_tweets.append(new_tweets['statuses'][x]['text'])
    
        oldest = tweet_id - 1
    
    twitter_data = pd.DataFrame({'id':tweet_ids, 'timestamp':timestamp, 'tweets':public_tweets})
    twitter_data = twitter_data.sort_values('id', ascending = False)
    return(twitter_data)

### Create Sentiment Analysis using Vader Sentiment

In [79]:
def sentiment(twitter_data):
    #create list to hold sentiments
    compound_list = []
    sentiments = []
    
    count = 0
    
    for tweet in twitter_data['tweets']:
        results = analyzer.polarity_scores(tweet)
        compound = results["compound"]
        
        time = twitter_data['timestamp'][count]
        new_time = convert_time(time)
        
        sentiments.append({"Date": new_time, 
                           "Compound": compound,
                           "Text": tweet,
                           })
        count = count + 1
            

            
    #Create a dataframe from dict sentiments
    df = pd.DataFrame.from_dict(sentiments, dtype = str)
    df = df.sort_values(by = 'Date', ascending = True).reset_index(drop = True)
    return(df)

### Create Module to convert time stamp

In [65]:
def convert_time (time):
    new_time = datetime.datetime.strptime(time, "%a %b %d %H:%M:%S %z %Y").timestamp()
    new_time = datetime.datetime.fromtimestamp(new_time).strftime('%Y-%m-%d %H:%M:%S')
    return(new_time)

In [80]:
twitter_data = twttr_data()
sentiment_pd = sentiment(twitter_data)

#Save dataframe into csv
sentiment_pd.to_csv('../Data_Sources/twitter_sentiments.csv')
twitter_data.to_csv('../Data_Sources/twitter_data.csv')

In [78]:
sentiment_pd

,Compound,Date,Text
0,0.5106,2018-07-16 03:43:19,Bitcoin Cloud Mining:\nFree Bitcoin Mining - a...
1,-0.0498,2018-07-15 11:10:51,RT @BibBoyne: It would really suck if you had ...
2,0.0,2018-07-13 03:23:36,#BTC #BTCUSD And we RUN! https://t.co/Fn3tXMkraZ
3,0.5563,2018-07-13 03:23:36,#BTC #BTCUSD Minor retarcement to 6480-6450.. ...
4,0.0,2018-07-13 03:23:36,#BTC #BTCUSD #IOTA #MIOTA #EOS still #Bullish....
5,0.0,2018-07-12 08:57:24,RT @Machoil_tokens: Airdrop starts soon\n\nNam...
6,0.5106,2018-07-11 13:41:54,RT @1234Revooo: Free Bitcoin Mining - an inves...
7,0.0,2018-07-09 09:15:21,potential reverse head and shoulders forming. ...
8,0.0,2018-07-08 15:02:52,RT @crypto_rand: Weekly $BTCUSD - $BTC chart:\...
9,-0.1027,2018-07-08 07:43:51,#BTCUSD D1 TR:\n- Would like to see a consolid...
